# Hadamard Walk Simulation

Placeholder notebook for implementing the Hadamard quantum walk using Qiskit.

This will serve as part of the quantum analysis and comparison in the project.

In [ ]:
import http.server, socketserver, threading, numpy as np, matplotlib.pyplot as plt, time
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.visualization import plot_histogram
from scipy.stats import binom, norm

PORT = 8000

def start_server():
    Handler = http.server.SimpleHTTPRequestHandler
    httpd = socketserver.TCPServer(("", PORT), Handler)
    print(f"Serving at http://localhost:{PORT}")
    httpd.serve_forever()

thread = threading.Thread(target=start_server)
thread.daemon = True
thread.start()

# Styling for plots
plt.style.use('seaborn-v0_8')  # Use Seaborn style

In [ ]:
from qiskit.compiler import transpile

# Parameters
num_steps = 100
num_pos_qubits = num_steps #int(np.ceil(np.log2(2 * num_steps + 1)))  # Enough for -4 to 3; increase for larger t
coin_qubit = num_pos_qubits
total_qubits = num_pos_qubits + 1
pos_qubits = list(range(num_pos_qubits))  # List for safety

def add_increment(pos_qubits):
    """Build +1 increment circuit."""
    num_bits = len(pos_qubits)
    inc_circ = QuantumCircuit(num_bits, name='Increment')
    inc_circ.x(pos_qubits[0])  # Flip LSB
    for i in range(1, num_bits):
        inc_circ.mcx(pos_qubits[:i], pos_qubits[i])  # Carry propagation
    return inc_circ

def add_decrement(pos_qubits):
    """-1: Inverse of increment (adjoint)."""
    num_bits = len(pos_qubits)
    dec_circ = QuantumCircuit(num_bits, name='Decrement')
    for i in range(num_bits - 1, 0, -1):
        dec_circ.mcx(pos_qubits[:i], pos_qubits[i])  # Reverse carry
    dec_circ.x(pos_qubits[0])  # Flip LSB last
    return dec_circ

# Main circuit
qc = QuantumCircuit(total_qubits, num_pos_qubits)  # Classical bits for position measure

# Initial state: H on coin, phase for symmetry
qc.h(coin_qubit)
qc.p(np.pi / 2, coin_qubit)  # i phase on |R>

for _ in range(num_steps):
    # Coin flip: H
    qc.h(coin_qubit)
    # Controlled increment (+1 if coin=1 |R>)
    inc_gate = add_increment(pos_qubits).to_gate().control(1)
    qc.append(inc_gate, [coin_qubit] + pos_qubits)
    
    # Controlled decrement (-1 if coin=0 |L>): Invert coin, control on 1 (now 0 original), revert
    qc.x(coin_qubit)
    dec_gate = add_decrement(pos_qubits).to_gate().control(1)
    qc.append(dec_gate, [coin_qubit] + pos_qubits)
    qc.x(coin_qubit)

# Measure position qubits
qc.measure(range(num_pos_qubits), range(num_pos_qubits))

# Transpile to decompose custom gates
qc_transpiled = transpile(qc, basis_gates=['u1', 'u2', 'u3', 'cx'], optimization_level=1)

qc.draw(output='mpl')

In [ ]:
aer_sim = AerSimulator().run(qc_transpiled,shots=10000)
counts = aer_sim.result().get_counts(qc_transpiled)

# Post-process: Convert binary to signed integers (two's complement)
def bin_to_signed(bin_str, num_bits):
    val = int(bin_str, 2)
    if val >= 2**(num_bits - 1):
        val -= 2**num_bits
    return val

signed_counts = {bin_to_signed(k, num_pos_qubits): v / 10000 for k, v in counts.items()}

# Custom plot for pitch
positions = np.arange(-num_pos_qubits, num_pos_qubits + 1)  # All possible for t=2
probs = [signed_counts.get(pos, 0) for pos in positions]
plt.bar(positions, probs, color='blue', label='Quantum (Ballistic)')

# Classical Gaussian overlay
classical_p = binom.pmf(np.arange(num_steps + 1), num_steps, 0.5)
classical_positions = np.arange(-num_steps//2, num_steps//2 + 1)
plt.plot(classical_positions, classical_p / sum(classical_p), 'r--', label='Classical (Gaussian)')

plt.title(f"Hadamard Quantum Walk ({num_steps} Steps) - Wave PDE Sim")
plt.xlabel("Position")
plt.ylabel("Probability")
plt.legend()
plt.show()

print("Signed Probabilities:", signed_counts)